<h1>Introduction </h1>
In this notebook , i'll try to solve this awesome ecdsa challenge , where i will try to figure out how to break a vulnerable ecdsa implementation .
<h2>prerequisites</h2>
<ul>
    <li> ecdsa</li>
    <li>lattices and hidden number problem</li>
</ul>

<h2> Challenge Description </h2>
the challenge was talking about a signature service , where you can sign any message you want . and here's the source code :

In [ ]:
#!/usr/bin/python 
import random 
import ecdsa
import hashlib
import time
from Crypto.Util.number import inverse
BITS_LENGTH = 16
MOD = 2**9

curve = ecdsa.NIST256p
G = curve.generator
n = curve.order

# make it a dict a key and description 
games = {
    "Batman: Arkham Origins":"Live like Batman and Protect Gotham City from the evil !",
    "GTA IV":"Welcome To Liberty City A place where you have to do some shitty work to live",
    "Uncharted 2":"After being tracked down by Harry Flynn, Nathan Drake goes on a quest for Marco Polo's lost fleet. However when things take an unexpected turn for the worst, Drake must rely on those closest to him in order to find the Cintomani Stone.",
    "IngeHack 2": "You never know what you are missing : ) "
}

def generate_random():
    r = random.randint(2**BITS_LENGTH,2**(BITS_LENGTH+1))
    return r+2**BITS_LENGTH  % MOD 

def print_intro(public_badge):
    print("==================   Welcome To IngeHack Station  ===================")
    print("To ensure the security of our station we are using a special ***secure*** system")
    print("Here is your public badge of the day ({},{})".format(int(public_badge.x()), int(public_badge.y())))
    print("Our system has proven to be unbreakable ! Don't waste your time and enjoy playing")

def show_welcome_menu():
    print("===========================================================")
    print("===============    Welcome Screen  ========================")
    print("===========================================================")
    print("To Start of you have the possibility to either play a game or get a digital Autograph from our Awesome Team :) ")
    print("1 - Play Game")
    print("2 - Get An Autograph")


def show_game_menu():
    print("===========================================================")
    print("=================    Game List  ===========================")
    print("===========================================================")
    print("Select the game number you want to play with ")
    for iterator, game in enumerate(games.items()):
        print(" Game: {} ===>  {} -  {}".format(iterator+1,game[0], game[1]))
    

def play_game(game_number,badge):
    # premium game 
    if game_number == 4:
        print("This is a PREMIUM Game you need to prove you are a premium member by showing us your private badge")
        priv_badge = int(input("=> "))
        if priv_badge == badge:
            print(open("flag.txt",'r').read())
        else:
            print("Hacker Detected ! Good Try: ) BYE")
    else:
        game_name = list(games.items())[game_number-1][0]
        print("...... Launching Game {} .......".format(game_name))
        time.sleep(1)
        print("ENJOY PLAYING {} ! BY IngehackStation ".format(game_name))

def sign_autograph(priv_badge):
    m1 = input("What do you want to be signed ? : ").encode()
    h1 = int(hashlib.sha256(m1).hexdigest(),16)
    done = False
    while not done:
        k = generate_random()
        P = k*G
        r = P.x()%n
        if r == 0:
            continue
        s = inverse(k,n)*(h1+r*priv_badge)%n
        if s == 0:
            continue
        done = True
    print("Here is your autograph (r,s) go show it to your friends ({},{})".format(r,s))


def main():
    private_badge = random.randint(1,n-1)
    public_badge = G*private_badge
    print_intro(public_badge)
    while True:
        show_welcome_menu()
        mode = int(input("> ")) 
        assert  1<=mode<=2
        if mode ==1 :
            show_game_menu()
            game_number = int(input("> "))
            assert 1<=game_number<=len(games)
            play_game(game_number,private_badge)
            exit(0)
        else:
            sign_autograph(private_badge)


if __name__ == '__main__':
    main()

<h2>Vulnerability </h2>
let : <ul>
    <li> h is the hash of a message m.</li>
    <li>G and n are the generator and the order of the curve.</li>
    <li>Public_key (P = d* G) , d in our challenge is the secret badge.</li>
    <li>(r,s) is the signature where r =P.x * (k*G) and s = $k^{-1}(h + rd) [n] $. </li>
    <li> k is supposed to be a random number , but here isn't the case and this is the main vulnerability. </li>
</ul>

now , if we have a set of signature pairs $(r_{i}, s_{i})$ we can construct a system of equations :
<ul>
    <li> $k_{1} -s_{1}^{-1}r_{1}d - s_{1}^{-1}h_{1} = 0 .$ </li>
   <li> $k_{2} -s_{2}^{-1}r_{2}d - s_{2}^{-1}h_{2} = 0 .$</li>
    <br>
      <li> $ k_{p} -s_{p}^{-1}r_{p}d - s_{p}^{-1}h_{p} = 0 $.</li>
</ul> 
and if the $k_{i}$ are small enough we can transform the problem to a short vector problem and we solve it using 
lattice's reduction . it's well explained in this blog
<a href="https://blog.trailofbits.com/2020/06/11/ecdsa-handle-with-care/" > Trail of Bits: ECDSA - Handle With Care. </a>
<br>
2 signature pairs are enough to solve the challenge , because| k1 - k2 | < 2** 9.

In [3]:
from sage.all import *
import ecdsa
import hashlib
from Crypto.Util.number import inverse
BITS_LENGTH = 16
MOD = 2**9

curve = ecdsa.NIST256p
G = curve.generator

h1=int(hashlib.sha256(b'a').hexdigest(), 16)
h2=int(hashlib.sha256(b'aa').hexdigest(), 16)

r1,s1 = (73735287451845428422627325012062020146744800091212920463561388407158065324976,91727460904042223407623983820644678889669951545658886799446586687680687213254)

r2,s2 = (46017201822517544616455798268827749280838479307905422188272891878735792906486,60232535732133042959781511463911920764520409127953061272213644308117568949873)
n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
pub_key = (110175056809402127424336102141813331735275980640325726248716323208474482955879,64584335676498201675073978604310520326652022090135849093873771151765233383023)

s1_inv = inverse(s1, n)
s2_inv = inverse(s2, n)


In [20]:
M =Matrix(QQ,4,4)

M[0] = [n,0,0,0]
M[1] = [0,n,0,0]
M[2] = [r1*s1_inv, r2*s2_inv , QQ(2**9)/ QQ(n) , 0]
M[3] = [h1*s1_inv, h2*s2_inv, 0, 2**9]
print(f"M : {M}")
M = M.LLL(delta = 0.75)
print(f"M after reduction: {M}")
#get the shortest vector
for v in M:
        if v[-1] == 2**9:
            v_short = v
            break
k0 = v_short[0]
    

d = inverse(r1, n) * (k0*s1 - h1) % n 
print(f" the secret badge  : {d}")

M : [                                                                            115792089210356248762697446949407573529996955224135760342422259061068512044369                                                                                                                                                          0                                                                                                                                                          0                                                                                                                                                          0]
[                                                                                                                                                         0                                                                             115792089210356248762697446949407573529996955224135760342422259061068512044369                                                                